# ASTROGET

In [1]:
__author__ = 'Steve Pothier <steve.pothier@noirlab.edu>'
__version__ = '20230831' # yyyymmdd; 
__keywords__ = ['HowTo', 'astronomy', 'tutorial']

This notebook demonstrates using the `astroget` package to get metadata and pixel data from the [NOIRLab Astro Data Archive](https://astroarchive.noirlab.edu/).

## Table of contents
* [Goals & Summary](#goalssummary)
* [Imports and setup](#imports)
* [Discover Images](#discover)
* [Get HDU](#gethdu)
* [Get cutout of object](#cutout)
* [Generate photo album](#album)
* [Find all Messier objects](#messier)
* [Save Messier snapshots](#save_album)

<a class="anchor" id="goals"></a>
## Goals & Summary 
Demonstrate the use of the `astroget` package to get metadata and pixel data from the [NOIRLab Astro Data Archive](https://astroarchive.noirlab.edu/). 
- Discovery: Search for matching metadata and return metadata records.
- Find images containing regions of sky (SIA)
- Retrieve images
  + Full FITS file
  + Single HDU of FITS file
  + Cutout of rectangular region of one HDU of a FITS file (as a new FITS file)


<a class="anchor" id="imports"></a>
## Imports and Setup

In [2]:
from pprint import pformat as pf
import os.path
from importlib import reload
from collections import defaultdict
from datetime import datetime
import warnings

import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy import units as u
import numpy as np
import pandas as pd

class StopExecution(Exception):
    def _render_traceback_(self):
        pass

# Suppress astropy warnings such as:
# WARNING: The following header keyword is invalid or follows an unrecognized ...
from astropy.utils.exceptions import AstropyWarning
warnings.simplefilter('ignore', category=AstropyWarning)

# %matplotlib inline
# requires installing ipympl
%matplotlib widget
plt.rcParams['font.size'] = 14

<a class="anchor" id="install"></a>
## Install the most recent version of the `astroget`:
*NOTE: After installing the most recent version, please restart your kernel.*

In [3]:
#!pip install -U astroget
#!pip install astroget==0.0.4a1.dev1
!pip install astroget==0.0.4a2.dev3
import astroget.client
from astroget.utils import tic,toc

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 KB 2.7 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: astroget
    Found existing installation: astroget 0.0.3
    Uninstalling astroget-0.0.3:
      Successfully uninstalled astroget-0.0.3


FileNotFoundError: [Errno 2] No such file or directory: 'pyproject.toml'

In [ ]:
reload(astroget.client)
print(f'Run started: {str(datetime.now())}')
# Server used must support client.cutout() method !!!
#server='https://astroarchive.noirlab.edu'     # Public server, HDUs>=408,118,252 Files>=18,731,991
server='https://marsnat1-pat.csdc.noirlab.edu' # Test Server,   HDUs>=26,246,808  Files>=711,608
#server='http://localhost:8060'                # Dev Server,    HDUs>=1,216
client = astroget.client.CsdcClient(url=server, show_curl=True)
client

# Basic Use of `astroget` for Cutouts
- Discover list of HDUs that contain pixels we want
  + get/display some metadata in pandas
  + retrieve/display a full matching HDU
- Cutout from HDU and display
- Cutout subimages from multiple HDUs and save FITS (appropriately named)

<a class="anchor" id="discover"></a>
## Find image/HDU of an object
Get position from name. Use additional constraints on the FITS files selected.
Display one of them.

Each HDU record contains `ra` and `dec` fields. Each of those is a RANGE. The Target for our cutout is given by an RA/DEC center and a size.  We can construct RA and DEC ranges for the Target (RA +/- width, DEC +/- height) We want only the HDUs that completely contain the Target. This kind of query can be done against our PostgreSQL 13 database using a "@>" ("contains") range operator.

   E.G. *ra @> target_ra*

Unfortunately, the syntax in FIND does not current support this. Fortunately, VOHDU does!

In [ ]:
# Discover image(s) of object using VOHDU
#fname='/net/archive/pipe/20170803/ct4m/2017B-0951/c4d_170804_050615_oow_r_v1.fits.fz'
#obj_name = 'M65' #65,(77,96)
obj_name = 'M54'  # ingested into DEV
obj_coord = SkyCoord.from_name(obj_name)
ra = obj_coord.ra.degree
dec = obj_coord.dec.degree

# default search using Overlap (essentially Postgres "&&" range operator)
# Use very small size to approximate "obj center in HDU"
#search_size = 0.0001 # radius in degrees
search_size = 0.05

In [ ]:
print(f'We will search {client.find(count=True,filerec=False).records[0]["count"]:,} HDUs.')

In [ ]:
# prod_type=image
# proc_type=stack,stacked
found = client.vohdu((ra,dec), search_size,  # position=(ra,dec), size(in decimal degrees)
            instrument='decam', obs_type='object', proc_type='instcal',
            VERB=3, limit=None, verbose=False)

assert found.count > 0, 'Must have some objects to use CUTOUT'
print(f'Found {found.count} matches of object {obj_name} at position: ra,dec={(ra,dec)} using client.vohdu()')


In [ ]:
hdus = [[rec[k] for k in ['md5sum','hdu_idx']] for rec in found.records]
targets = [[fid, hdu, ra, dec] for (fid,hdu) in hdus]
targets

In [ ]:
recidx = 0
rec = found.records[recidx]
print(f'Show 1 of {found.count} records. rec[{recidx}] = ')
print(f'{pf(rec)}')

md5 = rec['md5sum']
hduidx = rec['hdu_idx']
hduurl = rec['url']  

### API needs improvement; DISABLED
For FIND to be adequate for our needs, we need to be able to do something like:

`client.find(outfileds=[...], constraints={'hdu:ra': [target_ra, 'contains']})`

where 'ra' refers to the HDU RA field (defined as a range) and 'target_ra' refers to the range of RA we want in our cutout meaning numrange(min_target_ra, max_target_ra)

<a class="anchor" id="gethdu"></a>
## Get and Display one HDU

In [ ]:
# Make sure archive file is valid.  Normally this will always be true, but we 
# are recovering from a "mass storage" outage.
assert client.fitscheck(md5)['valid'], f"Invalid FITS file {rec['archive_filename']}"

# Image might be prioprietary. If so, using api/retrieve URL will error. 
# (But its possible to provide credentials...)
# API should provide more helpful error messages for exceptions: timeout, proprietary !!!
header0 = fits.getheader(hduurl, 0) # hdu0=Primary, hdu1=image
image_data1, header1 = fits.getdata(hduurl, 1, header=True) # hdu0=Primary, hdu1=image
print(f'HDU size in pixels = {image_data1.shape}')

%matplotlib widget
plt.imshow(image_data1, origin='lower', cmap='gray_r')

<a class="anchor" id="cutout"></a>
## Cutout subimage from HDU

In [ ]:
#subimage_size = 800
subimage_size = 150
print(f'Get subimage centered at ra,dec={(ra,dec)} size {subimage_size}. From image md5,hduidx=({md5}, {hduidx})')
# get FITS file
subimage = client.cutout(ra, dec, subimage_size, md5, hduidx, verbose=True)
print(f'subimage as been cutout into local FITS file: {subimage}')

image_data = fits.getdata(subimage)
plt.figure()
plt.imshow(image_data, origin='lower', cmap='gray_r')

<a class="anchor" id="cutouts"></a>
# Get a BATCH of cutouts

In [ ]:
print("Stopping execution to avoid long running search. "
      "Continue manually if you want.")
raise StopExecution

<a class="anchor" id="discoverBatch"></a>
## Find image/HDU of an object
Get position from name. Use additional constraints on the FITS files selected.

In [ ]:
targets

In [ ]:
url = client.cutouts(150, targets, wait=True, verbose=True)
print(f'Cutouts are available for download from {url}')

In [ ]:
recidx = 0
rec = found.records[recidx]
print(f'Show 1 of {found.count} records. rec[{recidx}] = ')
print(f'{pf(rec)}')

md5 = rec['md5sum']
hduidx = rec['hdu_idx']
hduurl = rec['url']  

In [ ]:
# Some fields of all records
mykeys = {'archive_filename', 'md5sum', 'hdu_idx'}
[{k: rec[k] for k in mykeys} for rec in found.records]

Note: two HDUs match in m54-6.fits

<a class="anchor" id="cutoutBatch"></a>
## Cutout a batch of subimages corresponding to many HDUs

In [ ]:
subimage_size = 150
print(f'Get subimage centered at ra,dec={(ra,dec)} size {subimage_size}'
      f' from image many HDUs')
# get tarball of FITS files
targets = [[rec['md5sum'], rec['hdu_idx'], ra, dec] for rec in found.records]
url = client.cutouts(subimage_size, targets, wait=True, verbose=True)
print(f'Cutouts are available for download from {url}')

<a class="anchor" id="album"></a>
# Generate astronomy photo album
Given a list of astro objects, find images that contain them and extract a subimage of the object. We don't know how big the objects are so all subimages are the same size. That size might be too big or too small.

In [ ]:
print("Stopping execution to avoid long running loop. "
      "Continue manually if you want.")
raise StopExecution

<a class="anchor" id="messier"></a>
## Find All Images (HDUs)for Messier objects in the Astro Data Archive
For each object name: get its position and search full archive for HDUs that overlap a box around the object position. All 393 mil(26 mil) HDUs are searched.

In [ ]:
%%time
albumdir = os.path.expanduser('~/astro-album/')
search_size = 0.0001 # radius in degrees
subimage_size = 800 # square pixels

objs = defaultdict(list)
for n in range(1,110+1):
    name = f'M{n}'
    obj_coord = SkyCoord.from_name(name)
    ra = obj_coord.ra.degree
    dec = obj_coord.dec.degree
    
    found = client.vohdu((ra,dec), search_size, VERB=3, 
                        instrument='decam', obs_type='object', proc_type='instcal',
                        limit=None)
    print(f'Found {found.count} "{name}" objects',end='. ')
    print
    for rec in found.records:
        objs[name].append(rec)
files = set([(r['md5sum'],r['archive_filename']) for obrecs in objs.values() for r in obrecs])
print(f'\n\nFound at least one HDU for objects: {[k for k,v in objs.items() if len(v) > 0]}')

In [ ]:
badfiles = {'/net/archive/pipe/20130910/ct4m/2012B-0001/c4d_130911_034852_ooi_i_d2.fits.fz',
 '/net/archive/pipe/20160923/ct4m/2012B-0001/c4d_160924_064533_oow_r_v2.fits.fz',
 '/net/archive/pipe/20190607/ct4m/2019A-0305/c4d_190608_045632_ooi_g_v1.fits.fz',
 '/net/archive/pipeline/Q20150425/DEC15A/20150407/c4d_150410_033027_ooi_z_v1.fits.fz',
 '/net/archive/pipeline/Q20150617/DEC15A/20150611/c4d_150613_092917_ooi_g_v1.fits.fz',
 '/net/archive/pipeline/Q20150719/DEC15A/20150714/c4d_150715_071229_ood_g_v1.fits.fz',
 '/net/archive/pipeline/Q20150918/REQ13B/HETDEX/c4d_131215_034511_oow_Y_v2.fits.fz',
 '/net/archive/pipeline/Q20160104/DEC15B/20151229/c4d_151230_063527_ood_i_v1.fits.fz',
 '/net/archive/pipeline/Q20160105/DEC15B/20151229/c4d_151230_053314_ood_r_v1.fits.fz',
 '/net/archive/pipeline/Q20160302/DEC16A/20160225/c4d_160302_022120_oow_g_v1.fits.fz',
 '/net/archive/pipeline/Q20160907/DEC16B/20160831/c4d_160906_034258_oow_VR_v1.fits.fz',
 '/net/archive/pipeline/Q20170111/DEC16B/20170102/c4d_170107_080204_oow_z_v1.fits.fz',
 '/net/archive/pipeline/Q20170111/DEC16B/20170102/c4d_170107_081751_oow_u_v1.fits.fz',
 '/net/archive/pipeline/Q20170111/DEC16B/20170102/c4d_170108_081653_oow_u_v1.fits.fz'}

In [ ]:
len(badfiles), len(files)

<a class="anchor" id="save_album"></a>
## Save local subimages for all Messier objects we found
Store all FITS subimates in a single directory using a file name that contains the Messier object name.
Each `client.cutout()` call makes a web-service call which: reads the HDU image data and metadata, extracts the cutout into a new tempory FITS file that contains the original Primary HDU and Extension=1 HDU that contains the submage (with WCS), and streams the temporary file in the HTTP response.  The client reads the response and saves it to a local file.

In [ ]:
%%time
aa = '~/astro-album/'
albumdir = os.path.expanduser(aa)
count = 0
tic()
for name,v in objs.items():
    total = len(v)
    for idx,rec in enumerate(v):
        print(f'Create cutout for {name}[{idx}/{total}]', end='...')
        
        #!if name=='M2' and idx==3:
        #!    continue
        if rec['archive_filename'] in badfiles:
            print('skipped', end='...')
            continue

        obj_coord = SkyCoord.from_name(name)
        ra = obj_coord.ra.degree
        dec = obj_coord.dec.degree
        outfile = f'{albumdir}/subimage_{name}_{idx}.fits'
        try:
            client.cutout(ra, dec, subimage_size, 
                          #rec['url'], 
                          rec['md5sum'], rec['hdu_idx'],
                          outfile=outfile)
            count += 1
        except Exception as err:
            print(f'\nFailed cutout for {name}[{idx}]; {err}')
            continue
elapsed = toc()
print('All Done!\n')
print(f'Created and retrieved {count} cutouts in {elapsed:0.2f} seconds. ({count/elapsed:0.1f} cutouts/sec)') # 0.4
#!print(f'You might want to view all files {aa} in  with QFitsView and turn on "blink" to get a slide show.')
print(f'\nYou might want to view all files in {aa} with ds9 (SAOImageDS9) and turn on "blink" to get a slide show.'
      f'\n  EXAMPLES: '
      f'\n  ds9 {aa}*.fits -blink'
      f'\n  ds9 {aa}*.fits -blink interval 0.1 -blink  # one frame every 0.1 seconds\n'
     )

In [ ]:
aa = '~/astro-album/'

In [ ]:
!ds9 {aa}*.fits -blink interval 0.2 -blink